<a href="https://colab.research.google.com/github/santiagogonzalezc/ML_Tumor/blob/Aura/BrainTumorProject_dicom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Brain-Tumor-Progression

This collection includes datasets from 20 subjects with primary newly diagnosed glioblastoma who were treated with surgery and standard concomitant chemo-radiation therapy (CRT) 
followed by adjuvant chemotherapy.  Two MRI exams are included for each patient: within 90 days following CRT completion and at progression (determined clinically, and based on 
a combination of clinical performance and/or imaging findings, and punctuated by a change in treatment or intervention). 

All image sets are in DICOM format and contain T1w (pre and post-contrast agent), FLAIR, T2w, ADC, normalized cerebral blood flow, normalized relative cerebral blood volume, 
standardized relative cerebral blood volume, and binary tumor masks (generated using T1w images).

The perfusion images were generated from dynamic susceptibility contrast (GRE-EPI DSC) imaging following a preload of contrast agent.
All of the series are co-registered with the T1+C images.  The intent of this dataset is for assessing deep learning algorithm performance to predict tumor progression.

https://wiki.cancerimagingarchive.net/display/Public/Brain-Tumor-Progression#3394811983c589667d0448b7be8e7831cbdcefa6


#### Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Install and import libraries

In [ ]:
!pip install -q pydicom
!pip install -q nipype
!pip install -q nilearn
!apt-get install -q dcm2niix
!apt-get install -q pigz

%matplotlib inline

import os
import pydicom 
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from nipype.interfaces import fsl
from nilearn.plotting import plot_anat
from nipype.interfaces.dcm2nii import Dcm2niix

#### Convert DICOM to NIfTI

In [ ]:
def dcm2nii_conv(folder_name,path,path_nii,patient,session,sequence):
  converter = Dcm2niix()
  if sequence != '.DS_Store': # Avoid troubles with 'invisible' files
    os.makedirs(path_nii+folder_name+patient,exist_ok=True)
    converter.inputs.source_dir = os.path.abspath(path+patient+'/'+session+'/'+sequence)
    converter.inputs.output_dir = os.path.abspath(path_nii+folder_name+'/'+patient)
    converter.inputs.single_file= True
    converter.id_in_filename= True
    converter.source_in_filename= True
    converter.cmdline
    print(converter.cmdline)
    converter.run()
  return print('success')

path = '/content/drive/MyDrive/DL/Brain-Tumor-Progression/' # Ubicación de los datos dicom de los sujetos
path_nii = '/content/drive/MyDrive/DL/test2/'# Ubicación para los datos anonimizados
files = os.listdir(path)
files.sort()
for patient in files:
  sessions = os.listdir(path+patient)
  for session in sessions:
    sequences = os.listdir(path+patient+'/'+session)
    for sequence in sequences:
      if "T1prereg" in sequence:
        # Load DICOM files, convert to nifti using Dcm2niix()
        folder_name = 'Brain-Tumor-Progression-nii/T1/'
        dcm2nii_conv(folder_name,path,path_nii,patient,session,sequence)
      elif "MaskTumor" in sequence:
        folder_name = 'Brain-Tumor-Progression-nii/MaskTumor/'
        dcm2nii_conv(folder_name,path,path_nii,patient,session,sequence)

dcm2niix -b y -z y -x n -t n -m n -o /content/drive/MyDrive/DL/test2/Brain-Tumor-Progression-nii/MaskTumor/PGBM-001 -s y -v n /content/drive/MyDrive/DL/Brain-Tumor-Progression/PGBM-001/04-02-1992-FH-HEADBrainProtocols-79896/5388.000000-MaskTumor-63335
210227-15:17:59,155 nipype.interface INFO:
	 stdout 2021-02-27T15:17:59.155650:Chris Rorden's dcm2niiX version v1.0.20171215 (OpenJPEG build) GCC7.3.0 (64-bit Linux)
210227-15:17:59,158 nipype.interface INFO:
	 stdout 2021-02-27T15:17:59.155650:Found 24 DICOM image(s)
210227-15:17:59,160 nipype.interface INFO:
	 stdout 2021-02-27T15:17:59.155650:Convert 24 DICOM as /content/drive/MyDrive/DL/test2/Brain-Tumor-Progression-nii/MaskTumor/PGBM-001/5388.000000-MaskTumor-63335_B_T1_AXIAL_SE_+_19920402085321_5388 (260x320x24x1)
210227-15:17:59,227 nipype.interface INFO:
	 stdout 2021-02-27T15:17:59.227556:compress: "/usr/bin/pigz" -n -f -6 "/content/drive/MyDrive/DL/test2/Brain-Tumor-Progression-nii/MaskTumor/PGBM-001/5388.000000-MaskTumor-63335_

## Save nifti files in a folder, without .json

In [ ]:
#read_nii = path_nii+'Brain-Tumor-Progression-nii/'
read_nii = '/content/drive/MyDrive/DL/test2/Brain-Tumor-Progression-nii/'
folders = os.listdir(read_nii)
for folder in folders:
  patients = os.listdir(read_nii+folder)
  for patient in patients:
    images = os.listdir(read_nii+folder+'/'+patient)
    for img in images:
      os.makedirs(read_nii+'/nii/'+folder,exist_ok=True)
      if img.endswith('.nii.gz'):
        os.chdir(read_nii+'/nii/'+folder+'/')
        data = nib.load(read_nii+folder+'/'+patient+'/'+img)
        nii_data=data.get_fdata()
        nii_afine=data.affine
        nii_image = nib.Nifti1Image(nii_data, nii_afine)
        nib.save(nii_image, img)

## Labels, Image and Mask

In [ ]:
path_nii = '/content/drive/MyDrive/DL/test2/Brain-Tumor-Progression-nii/nii/'# Ubicación para los datos anonimizados
type_img = os.listdir(path_nii)

labels = []
T1_data = []
mask_data = []

mask_files = os.listdir(path_nii+'MaskTumor')
T1_files = os.listdir(path_nii+'T1')

for T1 in T1_files:
  data_T1 = nib.load(path_nii+'T1/'+T1)
  T1_nii_data=data.get_fdata()
  for mask in mask_files:
    if T1[-27:-13] in mask: #Get the ID
      data_mask =  nib.load(path_nii+'MaskTumor/'+mask)
      mask_nii_data=data.get_fdata()
      label = 1

  labels.append(label)
  T1_data.append(T1_nii_data)
  mask_data.append(mask_nii_data)

brain_data = {'labels':labels,
              'T1':T1_data,
              'mask':mask_data}